In [1]:
import requests
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scraping_class
from matplotlib.dates import DateFormatter
import seaborn as sns
from bs4 import BeautifulSoup

In [2]:
plt.style.use('ggplot')
%matplotlib inline

SMALL_SIZE = 16
MEDIUM_SIZE = 18
BIGGER_SIZE = 20

plt.rc('font', size=SMALL_SIZE)          # controls default text sizes
plt.rc('axes', titlesize=SMALL_SIZE)     # fontsize of the axes title
plt.rc('axes', labelsize=MEDIUM_SIZE)    # fontsize of the x and y labels
plt.rc('xtick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('ytick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('legend', fontsize=SMALL_SIZE)    # legend fontsize
plt.rc('figure', titlesize=BIGGER_SIZE)  # fontsize of the figure title

In [3]:
logfile = 'oscar_winners_log.csv'
connector = scraping_class.Connector(logfile)

In [247]:

movies_data = []

for i in range(1,100, 100):
    url_nominees = 'https://www.imdb.com/search/title/?groups=oscar_best_picture_nominees&start=%s&ref_=adv_nxt' % str(i)
    url_winners = 'https://www.imdb.com/search/title/?count=100&groups=oscar_best_picture_winners&sort=year,desc&start=%s&ref_=nv_ch_osc' % str(i)

    response, call_id = connector.get(url_winners, 'get_winners')
    if response.ok:
        data = response.text
        soup = BeautifulSoup(data, "lxml")
        movies = soup.find_all('div', attrs={'lister-item-content'})


        for movie in movies:
            
            try:
                runtime = movie.find('p', attrs={'text-muted'}).find('span', attrs={'runtime'}).text
                genre   = movie.find('p', attrs={'text-muted'}).find('span', attrs={'genre'}).text.strip().split(', ')
                    
                
                idx, title, year = movie.find('h3', attrs={'lister-item-header'}).text.strip().split('\n')
                idx = idx.split('.')[0]
                link = movie.find('a')
                link = str(link).split('href="')[1].split('"')[0]
                link  = 'https://www.imdb.com' + link
                
                try:
                    if movie.find_all('span')[-2].text == 'Gross:':
                        gross = movie.find_all('span', attrs={'name':'nv'})[-1].text
                    else:
                        gross = np.nan
                except:
                    gross = np.nan
                
                try:
                    metascore = movie.find('span', attrs={'metascore'}).text
                except:
                    metascore = np.nan
                    
                job = movie.find_all('p')[2].text
                
                actors = [i.split('\n')[1] for i in job.split('Stars:')[1].split(',')]
                directors = [i.split(',')[0] for i in job.split('Stars:')[0].split('Director')[1].split(':\n')[1].split('\n')[:-2]]
                links_people = [str(i).split('href="')[1].split('"')[0] for i in movies[0].find_all('p')[2].find_all('a')]
                links_people = ["https://www.imdb.com/" + i + "awards" for i in links_people]
                
                try:
                    year = year.split('(I')[1]
                    try:
                        year = year.split('I) (')[1].split(')')[0]
                    except:
                        year = year.split(') (')[1].split(')')[0]
                except:
                    year = year.split('(')[1].split(')')[0]
                    
                movies_data.append([idx, title, year, runtime, genre, metascore, gross, link, directors, actors, links_people])
                
            except:
                NameError
            
    else:
        print('Response failed!')
        
df = pd.DataFrame(movies_data)
df.columns = ['index', 'title', 'year', 'runtime', 'genre', 'metascore', 'gross', 'link_movie', 'director', 'actors', 'link_people']

In [254]:
df

,index,title,year,runtime,genre,metascore,gross,link_movie,director,actors,link_people
0,1,Green Book,2018,130 min,"[Biography, Comedy, Drama]",69,$85.08M,https://www.imdb.com/title/tt6966692/,[Peter Farrelly],"[Viggo Mortensen, Mahershala Ali, Linda Cardel...","[https://www.imdb.com//name/nm0268380/awards, ..."
1,2,The Shape of Water,2017,123 min,"[Adventure, Drama, Fantasy]",87,$63.86M,https://www.imdb.com/title/tt5580390/,[Guillermo del Toro],"[Sally Hawkins, Octavia Spencer, Michael Shann...","[https://www.imdb.com//name/nm0268380/awards, ..."
2,3,Moonlight,2016,111 min,[Drama],99,$27.85M,https://www.imdb.com/title/tt4975722/,[Barry Jenkins],"[Mahershala Ali, Naomie Harris, Trevante Rhode...","[https://www.imdb.com//name/nm0268380/awards, ..."
3,4,Spotlight,2015,129 min,"[Biography, Crime, Drama]",93,$45.06M,https://www.imdb.com/title/tt1895587/,[Tom McCarthy],"[Mark Ruffalo, Michael Keaton, Rachel McAdams,...","[https://www.imdb.com//name/nm0268380/awards, ..."
4,5,Birdman or (The Unexpected Virtue of Ignorance),2014,119 min,"[Comedy, Drama]",87,$42.34M,https://www.imdb.com/title/tt2562232/,[Alejandro G. Iñárritu],"[Michael Keaton, Zach Galifianakis, Edward Nor...","[https://www.imdb.com//name/nm0268380/awards, ..."
5,6,12 Years a Slave,2013,134 min,"[Biography, Drama, History]",96,$56.67M,https://www.imdb.com/title/tt2024544/,[Steve McQueen],"[Chiwetel Ejiofor, Michael Kenneth Williams, M...","[https://www.imdb.com//name/nm0268380/awards, ..."
6,7,Operation Argo,2012,120 min,"[Biography, Drama, Thriller]",86,$136.03M,https://www.imdb.com/title/tt1024648/,[Ben Affleck],"[Ben Affleck, Bryan Cranston, John Goodman, Al...","[https://www.imdb.com//name/nm0268380/awards, ..."
7,8,The Artist,2011,100 min,"[Comedy, Drama, Romance]",89,$44.67M,https://www.imdb.com/title/tt1655442/,[Michel Hazanavicius],"[Jean Dujardin, Bérénice Bejo, John Goodman, J...","[https://www.imdb.com//name/nm0268380/awards, ..."
8,9,Kongens store tale,2010,118 min,"[Biography, Drama, History]",88,$138.80M,https://www.imdb.com/title/tt1504320/,[Tom Hooper],"[Colin Firth, Geoffrey Rush, Helena Bonham Car...","[https://www.imdb.com//name/nm0268380/awards, ..."
9,10,Slumdog Millionaire - Drengen der havde svar p...,2008,120 min,"[Drama, Romance]",86,$141.32M,https://www.imdb.com/title/tt1010048/,"[Danny Boyle, Loveleen Tandan]","[Dev Patel, Freida Pinto, Saurabh Shukla, Anil...","[https://www.imdb.com//name/nm0268380/awards, ..."


In [256]:
df.to_csv('oscar_nominees.csv')